In [2]:
# JPAULS IPYTHON NOTEBOOK

In [1]:
# Does the necessary imports!
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import time

In [2]:
# Gets main page's HTML
mainpagelink = 'http://racingchannel.com/results_archive.php'
mainpagehtml = requests.get(mainpagelink).text

In [3]:
# Gets list of racetracks, racetrackcodes, and creates a dict of the two (can flip the order if necessary)
racetrackshtmllst = BeautifulSoup(mainpagehtml, 'html.parser').body.table.findAll('tr')[4].findAll('td')
tracks = []
trackcodes = []
trackdatalinks = []
for tracklst in racetrackshtmllst:
    tracklinks = tracklst.findAll('a')
    for link in tracklinks:
        #print link.get('href').encode('ascii')
        trackdatalinks.append('http://racingchannel.com/'  + link.get('href').encode('ascii'))
        trackcodes.append(link.get('href')[10:13].encode('ascii'))
        tracks.append(link.text.encode('ascii'))
tracks[1] = 'Arlington Park'
# print tracks
# print '\n\n'
# print trackcodes
# print '\n\n'
trackcodedict = dict(zip(tracks, trackcodes))
tracklinkdict = dict(zip(tracks, trackdatalinks))
# print '\n\n'
trackinfodf = pd.DataFrame(index=tracks)
trackinfodf['code'] = trackcodes
trackinfodf['link'] = trackdatalinks
trackinfodf.head(5)

,code,link
Aqueduct,AQU,http://racingchannel.com//archives/AQU/default...
Arlington Park,ARL,http://racingchannel.com//archives/ARL/default...
Australian Racing,AU1,http://racingchannel.com//archives/AU1/default...
Australian Racing - B,AU2,http://racingchannel.com//archives/AU2/default...
Australian Racing - C,AU3,http://racingchannel.com//archives/AU3/default...


In [122]:
# Stores the html of all track info pages

#############################################################
## SKIP THIS IF TEMPDATA ALREADY HAS TRACKINFODF IN IT!!!! ##
#############################################################


htmloftracks = []
for link in trackdatalinks:
    htmloftracks.append(BeautifulSoup(requests.get(link).text))
    print link
    time.sleep(5)

trackinfodf['track_html'] = htmloftracks
trackinfodf.head(10)

# Stores track codes and links in csv
trackinfodf.to_csv('tempdata/trackinfo.csv', index_label='name')

http://racingchannel.com//archives/AQU/default.html
http://racingchannel.com//archives/ARL/default.html
http://racingchannel.com//archives/AU1/default.html
http://racingchannel.com//archives/AU2/default.html
http://racingchannel.com//archives/AU3/default.html
http://racingchannel.com//archives/BAY/default.html
http://racingchannel.com//archives/BEL/default.html
http://racingchannel.com//archives/BEU/default.html
http://racingchannel.com//archives/CAL/default.html
http://racingchannel.com//archives/CAM/default.html
http://racingchannel.com//archives/CAN/default.html
http://racingchannel.com//archives/CTN/default.html
http://racingchannel.com//archives/CHU/default.html
http://racingchannel.com//archives/COL/default.html
http://racingchannel.com//archives/DDN/default.html
http://racingchannel.com//archives/DEL/default.html
http://racingchannel.com//archives/DPK/default.html
http://racingchannel.com//archives/ELL/default.html
http://racingchannel.com//archives/EMD/default.html
http://racin

,code,link,track_html
Aqueduct,AQU,http://racingchannel.com//archives/AQU/default...,<html> <head><title>Aqueduct Results Archive</...
Arlington Park,ARL,http://racingchannel.com//archives/ARL/default...,<html> <head><title>Arlington Results Archive<...
Australian Racing,AU1,http://racingchannel.com//archives/AU1/default...,<html> <head><title>Australia - A Results Arch...
Australian Racing - B,AU2,http://racingchannel.com//archives/AU2/default...,<html> <head><title>Australia - B Results Arch...
Australian Racing - C,AU3,http://racingchannel.com//archives/AU3/default...,<html> <head><title> Results Archive</title></...
Bay Meadows,BAY,http://racingchannel.com//archives/BAY/default...,<html> <head><title>Bay Meadows Results Archiv...
Belmont Park,BEL,http://racingchannel.com//archives/BEL/default...,<html> <head><title>Belmont Results Archive</t...
Beulah Park,BEU,http://racingchannel.com//archives/BEU/default...,<html> <head><title>Beulah Results Archive</ti...
Calder Race Course,CAL,http://racingchannel.com//archives/CAL/default...,<html> <head><title>Calder Results Archive</ti...
Camarero,CAM,http://racingchannel.com//archives/CAM/default...,<html> <head><title> Results Archive</title></...


In [4]:
# Loads trackinfodf from the csv in tempdata
del trackinfodf
trackinfodf = pd.read_csv('tempdata/trackinfo.csv', index_col='name')
trackinfodf['track_html'] = trackinfodf['track_html'].apply(lambda h:BeautifulSoup(h, 'html.parser'))
trackinfodf.head(5)

,code,link,track_html
name,,,
Aqueduct,AQU,http://racingchannel.com//archives/AQU/default...,<html> <head><title>Aqueduct Results Archive</...
Arlington Park,ARL,http://racingchannel.com//archives/ARL/default...,<html> <head><title>Arlington Results Archive<...
Australian Racing,AU1,http://racingchannel.com//archives/AU1/default...,<html> <head><title>Australia - A Results Arch...
Australian Racing - B,AU2,http://racingchannel.com//archives/AU2/default...,<html> <head><title>Australia - B Results Arch...
Australian Racing - C,AU3,http://racingchannel.com//archives/AU3/default...,<html> <head><title> Results Archive</title></...


In [51]:
# For each track rip the 

tracksoups = trackinfodf['track_html']
yrmonthdicts_by_track = []
for soup in tracksoups:
    try:
        soup = soup.body.center.findAll('table')[1].tr.findAll('td')
    except:
        soup = soup.body.center.findAll('table')[0].tr.findAll('td')
    years = []
    yrmonths_for_track = []
    for col in soup:
        year = col.text.encode('ascii')[:4]
        years.append(year)
        months = [elem.get('href').encode('ascii')[19:21] for elem in col.findAll('a')]
        yrmonths_for_track.append({year: months})

    yrmonthdicts_by_track.append(yrmonths_for_track)
biggie = dict(zip(trackcodes,yrmonthdicts_by_track))
print biggie

{'EMD': [{'1998': ['08', '09']}, {'1999': ['04', '05', '06', '07', '08', '09']}, {'2000': ['04', '05', '06', '07', '08', '09']}, {'2001': ['04', '05', '06', '07', '08', '09']}, {'2002': ['04', '05', '06', '07', '08', '09']}, {'2003': ['04', '05', '06', '07', '08', '09']}, {'2004': ['04', '05', '06', '07', '08', '09']}, {'2005': ['04', '05', '06', '07', '08', '09', '10']}, {'2006': ['04', '05', '06', '07', '08', '09', '10']}, {'2007': ['04', '05', '06', '07', '08', '09']}, {'2008': ['04', '05', '06', '07', '08', '09']}, {'2009': ['04', '05', '06', '07', '08', '09']}, {'2010': ['04', '05', '06', '07', '08', '09']}, {'2011': ['04', '05', '06', '07', '08', '09']}, {'2012': ['04', '05', '06', '07', '08', '09']}, {'2013': ['04', '05', '06', '07', '08', '09']}, {'2014': ['04', '05', '06', '07', '08', '09']}, {'2015': ['04', '05', '06', '07', '08', '09']}], 'SAN': [{'1998': ['09', '10', '11', '12']}, {'1999': ['04', '09', '10', '11', '12']}, {'2000': ['04', '10', '11', '12']}, {'2001': ['01', 

In [126]:
%%time
tstorage = []
trackday_lists = []
for track in sorted(biggie.keys()):
    print track
    ystorage = []
    for yeardict in biggie[track]:
        mstorage = []
        for month in yeardict.values()[0]:
            monthhtml = BeautifulSoup(requests.get('http://racingchannel.com/archives/{}/{}/{}/default.html'\
                                         .format(track,yeardict.keys()[0],month)).text, 'html.parser').body.center
            try:
                soup = monthhtml.findAll('table')[1]
            except:
                soup = monthhtml.findAll('table')[0]
            days = [daytag.get('href')[29:31].encode('ascii') for daytag in soup.findAll('a')]
            mo = daytag.get('href')[19:21].encode('ascii')
            yr = daytag.get('href')[14:18].encode('ascii')
            trk = daytag.get('href')[22:25].encode('ascii')
            for day in days:
                trackday_lists.append([trk, yr, mo, day]) 
            time.sleep(2)
            #print 'ya'
        time.sleep(5)
    time.sleep(10)
    print len(trackday_lists)
print len(trackday_lists)

AQU


KeyboardInterrupt: 

In [ ]:
print len(trackday_lists)
daydf = pd.DataFrame(trackday_lists, columns=['track','year','month','day'])
daydf.to_csv('tempdata/daysscraped.csv')

In [4]:
lsttouse = ['AQU','1998','10','28']
magiclink = 'https://www.racingchannel.com/archives/{}/{}/{}/{}{}{}{}.HTM'\
    .format(inlst[0],inlst[1],inlst[2],inlst[0],inlst[1][2:],inlst[2],inlst[3])

racetest = requests.get(linktouse).text

soup = BeautifulSoup(racetest, 'html.parser').html.body

In [17]:
tables = soup.findAll('tr')
print len(tables)
for elem in tables[:6]:
    print elem

73
<tr>
<td colspan="2"><b><u>Race:</u></b></td>
<td width="20"><b><u>1</u></b></td>
<td align="RIGHT" width="60"><b><u>Win</u></b></td>
<td align="RIGHT" width="60"><b><u>Place</u></b></td>
<td align="RIGHT" width="60"><b><u>Show</u></b></td>
<td width="60"></td>
</tr>
<tr>
<td width="30"><b>1st</b></td>
<td width="20"><b>3</b></td>
<td align="LEFT" width="200"><b>CURVY IMAGE</b></td>
<td align="RIGHT" width="60">6.30</td>
<td align="RIGHT" width="60">3.80</td>
<td align="RIGHT" width="60">3.00</td>
</tr>
<tr>
<td><b>2nd</b></td>
<td><b>6</b></td>
<td><b>DYNABILITY</b></td>
<td align="RIGHT"></td>
<td align="RIGHT">5.90</td>
<td align="RIGHT">4.30</td>
</tr>
<tr>
<td><b>3rd</b></td>
<td><b>2</b></td>
<td><b>ALYDAR'S JET</b></td>
<td align="RIGHT"></td>
<td align="RIGHT"></td>
<td align="RIGHT">5.00</td>
</tr>
<tr>
<td colspan="3"><b>$2 Exacta</b></td>
<td align="CENTER" colspan="2" colstart="4"><b>3-6</b></td>
<td align="RIGHT">34.60</td>
</tr>
<tr>
<td colspan="2" colstart="3"><b>Ref

In [ ]:
# 1) first populate "inlsts" with all the trackkeys, years, months, and days
# 2) scrape each page for its html
# 3) rip through the html for the info we want
# 4) format the info into what we're feeling

In [ ]:
for stadium in stadiums:
    for year in years:
        